In [2]:
import random
import json
from pprint import pprint
import nltk
import re
import string
from nltk.tag.stanford import StanfordPOSTagger
from nltk.tag.stanford import StanfordNERTagger
from nltk.chunk import RegexpParser
import nertclient

#PART 0:
##Data Cleaning


We abtained the source data from (TODO Jordan)

We first eliminated TV shows and focused on only moviews.

Then we reduced the movies to ones with 10 or more IMDB reviews so that we only analyze movies with decent infleunce.

We then matched the IMDB movie titles with their wikipedia counterparts, and this turned out to be a challenge on its own. Since our IMDB dataset is larger then the wikipedia dataset, many movies have IMDB summaries but not a wikipedia summary. Our final database included movies that have IMDB summaries and/or wikipedia summaries.

Lastly, we matched our movies to their country data, and reduced our dataset to just US movies. It contains about 34,000 movies.

#PART 1: 
##Named Entity Recognition Tagging for Movie Summaries 

### import Standford POS Tagger and Stanford NER Tagger

In [67]:
#to run StanfordPostTagger and NERTagger, 
#first download these two packages from http://nlp.stanford.edu/software/CRF-NER.shtml 
#I saved the downloaded files in lib/
post = StanfordPOSTagger('lib/stanford-postagger-2014-08-27/models/english-bidirectional-distsim.tagger', 
                         'lib/stanford-postagger-2014-08-27/stanford-postagger.jar', 'utf-8') # doctest: +SKIP
# post.tag('What is the airspeed of an unladen swallow ?'.split()) # doctest: +SKIP
nert = StanfordNERTagger('lib/stanford-ner-2014-08-27/classifiers/english.all.3class.distsim.crf.ser.gz',
                         'lib/stanford-ner-2014-08-27/stanford-ner.jar', 'utf-8')

###Load in cleaned dataset of USA moviews from IMDB and Wikiperida

In [ ]:
with open('usamoviedict.array.json') as sample:
# with open('movies-with-roles-summaries.json') as sample:
    sample = json.load(sample)

###More data cleaning

In [69]:
# get rid of ([[ in the wikipidia summaries
for movie in sample:
    summaries_wiki = movie[1]['summaries_wikipedia']
    if len(summaries_wiki) > 0:
        summaries_wiki = re.sub(r'\s\(\[\[', ' ', summaries_wiki[0])
        movie[1]['summaries_wikipedia'][0] = summaries_wiki

###NER Tagging

In [71]:
#pos tagging using nltk.pos_tag
#ner tagging using stanford ner tagger
def ie_preprocess(document, lower='false', stage="pos"):
#     if stage == 'pos':
#         sentences = nltk.sent_tokenize(document)
#         sentences = [nltk.word_tokenize(sent) for sent in sentences]
#         if lower == 'false':
#             for sent in sentences:
#                 for i in range(len(sent)):
#                     if sent[i] != sent[i].lower():
#                         sent[i] = sent[i].lower()
#         sentences = [nltk.pos_tag(sent) for sent in sentences]
#         return sentences
    if stage == "ner":
        return nertclient.nertclient(document)

In [ ]:
# make stanford nert more accurate by seperating the last word
# in a sentence from the ending punctuation with a space.
def insert_space_before_punct(s):
    # return re.sub(r'([-\/#!$%\^&\*;:{}=\-_`~()])', r' \1', s)
    # s = re.sub(r'([-\/#$%\^&\*{}=\-_`~()])', r'', s)
    s = re.sub(r'([.,!?;:])', r' \1', s)
    return s

In [72]:
# pos + ner tag summaries and roles
for movie in sample:
    summaries_imdb = movie[1]["summaries_imdb"]
    summaries_wiki = movie[1]['summaries_wikipedia']
    roles = movie[1]['roles']
    if len(summaries_imdb) > 0:
        # sample[i][1]["summaries_imdb_pos"] = ie_preprocess(summaries_imdb[0])
        sample[i][1]["summaries_imdb_ner"] = ie_preprocess(insert_space_before_punct(summaries_imdb[0]), stage="ner")
        
    if len(summaries_wiki) > 0:
        # sample[i][1]["summaries_wikipedia_pos"] = ie_preprocess(summaries_wiki[0])
        sample[i][1]["summaries_wikipedia_ner"] = ie_preprocess(insert_space_before_punct(summaries_wiki[0]), stage='ner')


###Output NER tagged dataset as json file usamoviedict.sample.json

In [12]:
def set_default(obj):
    if isinstance(obj, set):
        d = {}
        for i in obj:
            d[i] = 1
        return d
        # return list(obj)
    raise TypeError

In [ ]:
with open(filename + '.sample.json', 'w') as outfile:
    json.dump(sample, outfile, ensure_ascii=False, default=set_default)
 

#PART 2: 
##Lift out character names from summaries, and associate the character names with names from the IMDB role list.

###Load in the previously NER tagged dataset

In [3]:
with open('usamoviedict.array.json.0.sample.json') as tagged_data:
    sample = json.load(tagged_data)

###Create a new master dictionary called "sum_and_char" containing character names and related info.

####First, add new fields "summaries_combined" and "summaries_combined_ner" in the dataset that stores wikipedia and IMDB summaries into a single string for each movie.

In [4]:
# combine wiki_ner and imdb_nerie
for movie in sample:
    combined = []
    combined_ner = ""
    if 'summaries_wikipedia_ner' in movie[1]:
        combined.extend(movie[1]['summaries_wikipedia'] )
        combined_ner = combined_ner + " " + (movie[1]['summaries_wikipedia_ner'] )
    if 'summaries_imdb_ner' in movie[1]:
        combined.extend(movie[1]["summaries_imdb"])
        combined_ner = combined_ner + " " + (movie[1]["summaries_imdb_ner"])
    movie[1]['summaries_combined'] = combined
    movie[1]['summaries_combined_ner'] = combined_ner

####Second, run a customized regex chuncker to lift out tagged names from combined summaries

Create a json file containing a list of names for the social security dataset, this file will be used in the person_extractor_and_replacer function. The resulting name_list.json contains 138,036 unique names (93,889 first names, 58,258 last names)

In [ ]:
with open('names.json') as data:
    first_names = json.load(data)
with open('surnames.json') as data2:
    last_names = json.load(data2)

name_list = []

for key, value in first_names.items():
    name_list.append(key)
for key, value in last_names.items():
    name_list.append(key)

def set_default(obj):
    if isinstance(obj, set):
        d = {}
        for i in obj:
            d[i] = 1
        return d
    raise TypeError
                    
with open("name_list.json", 'w') as outfile:
    json.dump(set(name_list), outfile, ensure_ascii=False, default=set_default)

In [15]:
#character name chunking.
#1. tag words in summaries that match the Social Security name dataset as "/PERSON"
#2. chunk together the consective "/PERSON" tagged words
#3. append to a chunk any number of capitalized word after a '/PERSON" tagged word (this helps capture the entirity
# of a name that spans multiple words. some of the words toward the end may not have been tagged correctly.)

def person_extractor_and_replacer(tagged_summary):
    with open('name_list.json', 'r') as json_str:
        ssn_names = json.load(json_str)

    words = tagged_summary.split()

    persons = []
    current_person = []

    for w in words:
        m = re.search('([^/]+)/([A-Z]+)', w)
        if m is not None:
            body = m.group(1)
            klass = m.group(2)
            if klass != "PERSON":
                if body in ssn_names:
                    klass = "PERSON"

            if klass == "PERSON":
                current_person.append(body)
                 #print(body, klass)
            elif len(current_person) > 0 and re.search('^[A-Z]', w) is not None:
                current_person.append(body)
            elif len(current_person) > 0:
                persons.append(" ".join(current_person))
                # persons.append(current_person)
                current_person = []
            # search for matches with social security name list
    return persons

In [7]:
# creating a new dict called sum_and_char to store summaries and corresponding characters
sum_and_char = {}
for movie in sample:
#     sorting entity_chuncking results into summaries and characters
    result = person_extractor_and_replacer(movie[1]['summaries_combined_ner'])
    movie_name = movie[0]
    if len(result) <= 0:
        char_data = {}
    else:
        char_data = set(result)
    sum_and_char[movie_name] = {"sum": movie[1]['summaries_combined'][0], \
                                "char_raw": char_data,\
                                "char_info" : {}}

###Filter character names, and establish the link between filtered character names from summaries and their counter parts in the roles list. Store results in the field char_info dict within the master dict

In [9]:
for movie in sample:
#     making a bag of words for roles
    for role in movie[1]['roles']:
        role_words = role['role']
        role_bag = re.split(r' |/|\'|\"', role_words)
        role['role_bag'] = (set(role_bag))

In [10]:
# filter char_raw against role list and link char_names with their names in the role list.
''' 
match words in char_raw against roles in role list. the list of actual character names is a subset of char_raw.
This is because some of the captured names are not characters (e.g. director, actor, spriris and saints). We filter
out the false names by only keeping char_raw names if at least one word in the name has a matched word in the role list.
If there is zero word match, discard. If there is at least one
match, keep.

char_info in sum_and_char use names in the role list as keys, and contains role gender, char names
found in summaries, and roles found in summaries

The matching selects the highest # of matches between a role and a character name. Ties are broken by selecting the role with
the fewest words.
'''
for key, value in (sum_and_char.items()):
    for movie in sample:
#     locate correct movie
        if movie[0] == key:
#             print()
#             print("--------", key, "--------------------------------------")
#             for each filted char name
            for name in value['char_raw']:
                max_count = 0
                max_role = None
                max_gen = None
#                 tie-breaker on # of words contained in a role_bag
                max_role_bag_len = 0

#                 tied_list = []
                name_split = name.split()
#                 for each role
                for role in movie[1]['roles']:
                    count = 0
#                   for each word in char name
#                   increment count by 1 every time a word in the char name appears in a role
                    for word in name_split:
                        if word in role['role_bag']:
                            count += 1
#                   select the role with the most counts!
                    if (count > max_count) or (count == max_count and len(role['role_bag']) < max_role_bag_len):
                        max_count = count
                        max_role = role['role']
                        max_gen = role['gender']
                        max_role_bag_len = len(role['role_bag'])
                
#                 After looping through all roles in the role list, decide which is the max role
                if max_count < 1:
#                     print(name, "discarded")
                    pass
                else:   
                    if max_role not in value['char_info']:
                        value['char_info'][max_role] = {'gender':max_gen, 'roles_found_in_sums':[], 'names_found_in_sums':[name]}
                    else:
                        value['char_info'][max_role]['names_found_in_sums'].append(name)

###Lastly, output the master dict sum_and_char as a json file

In [13]:
with open('sam_char_char.complete.v1.json', 'w') as outfile:
    json.dump(sum_and_char, outfile, ensure_ascii=False, default=set_default)
 

#PART 3: 
###Find roles for character names in summary!

#PART 4:
###Result Analysis